## DF vs DF_FineTune

- Use validation set to report results
- Note these are two different experiments with two different datasets

- (DONE) Test DF-detector vs. megadetector using DF-classifier
    - confidence_threshold = 0
    - Use full dataset
    - Precision/recall/F1 for animal/human/vehicle/empty (do wild boar deep verification)

- With megadetector, test DF-fine-tuned:
    - Dataset where megadetector detected an animal
    - Precision/recall/F1 for all species, plus total
    - 5 epochs, lr1e-5
    - 5 epochs, lr1e-5 & 5 epochs, wb loss